In [62]:
import pandas as pd
import json

basePath = "/Users/isratjahan/Desktop/gtfs_subway/"

stops = pd.read_csv(basePath + "stops.txt")
routes = pd.read_csv(basePath + "routes.txt")
trips = pd.read_csv(basePath + "trips.txt")
stop_times = pd.read_csv(basePath + "stop_times.txt")

subway_routes = routes[routes["route_type"] == 1].copy()

subway_trips = trips.merge(
    subway_routes[["route_id", "route_short_name"]],
    on="route_id",
    how="inner"
)

subway_stop_times = stop_times.merge(
    subway_trips[["trip_id", "route_short_name"]],
    on="trip_id",
    how="inner"
)

subway_stops = subway_stop_times.merge(
    stops[["stop_id", "stop_name", "parent_station"]],
    on="stop_id",
    how="left"
)

subway_stops["station_id"] = subway_stops["parent_station"].fillna(subway_stops["stop_id"])

def canonical_name(group):
    return group["stop_name"].value_counts().idxmax()

station_names = subway_stops.groupby("station_id").apply(canonical_name).rename("station_name")

station_routes = (
    subway_stops.groupby("station_id")["route_short_name"]
    .apply(lambda x: sorted(set(x.dropna())))
    .to_frame()
    .join(station_names, on="station_id")
)

mapping = {}
for _, row in station_routes.iterrows():
    mapping[row["station_name"]] = row["route_short_name"]

with open("station_to_lines.json", "w") as f:
    json.dump(mapping, f, indent=2, sort_keys=True)

print("Done")


Done


/var/folders/dy/xq75wbj1345d8bk_wpg5rl8r0000gn/T/ipykernel_30683/713476409.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  station_names = subway_stops.groupby("station_id").apply(canonical_name).rename("station_name")
